In [ ]:
TODO -> random forest?

In [1]:
import data_helpers
import os
import pandas as pd

load_main = True

main_cats = ['ActionName', 'Capability']
dataframes = {cat : {} for cat in main_cats} if load_main else {}

for cat in main_cats:
    path = os.getcwd() + "/Dataframes/" + cat + "/Processed/"
    ls = os.listdir(path)
    for file in ls:
        if (load_main and cat in file) or (not load_main and cat not in file):
            df = pd.read_csv(path + file, encoding='utf-8')

            df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
            df.drop(["a"], axis=1, inplace=True)

            df.rename({"Unnamed: 0.1":"a"}, axis="columns", inplace=True)
            df.drop(["a"], axis=1, inplace=True)

            df_type = file.split('-')[1]
            if load_main:
                dataframes[cat][df_type] = df
            else:
                sub_cat = file.split('-')[0]
                if sub_cat not in dataframes.keys():
                    dataframes[sub_cat] = {}
                dataframes[sub_cat][df_type] = df

if not load_main:
    main_cats = list(dataframes.keys())
main_cats

['ActionName', 'Capability']

In [4]:
if load_main:
    print(dataframes['ActionName']['Train'].columns)
else:
    print(dataframes['File']['Train'].columns)

Index(['label', 'text-rel', 'label_num', 'tokens', 'text-neigh', 'token',
       'text-rel-processed', 'text-neigh-processed', 'text-rel-tokens',
       'text-neigh-tokens'],
      dtype='object')


In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from clf_helpers import do_the_pipeline

def dummy(doc):
    return doc

In [4]:
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.metrics import classification_report, confusion_matrix

pipeline = Pipeline([          
    ('vectorizer',
     CountVectorizer(
         preprocessor=dummy, 
         ngram_range=(1, 4),
         analyzer='word',
         binary=False
     )),
    ('clf', 
     LogisticRegression(
         class_weight='balanced', # better results for imbalanced Capability - NoCapability
         solver='liblinear' # podobno: best for small datasets
     )
    )
])

do_the_pipeline(pipeline, main_cats, dataframes, accuracy=True, report=True, top_features=False)


>>> ActionName:

# RELATIONS
Vectorizer: W słowniku znajduje się 43659 różnych słów
- Zbiór treningowy:
	ACCURACY: 99.6117%
              precision    recall  f1-score   support

  ActionName       0.99      1.00      0.99      1154
NoActionName       1.00      0.99      1.00      2194

   micro avg       1.00      1.00      1.00      3348
   macro avg       0.99      1.00      1.00      3348
weighted avg       1.00      1.00      1.00      3348

- Zbiór testowy:
	ACCURACY: 77.2152%
              precision    recall  f1-score   support

  ActionName       0.66      0.68      0.67        80
NoActionName       0.83      0.82      0.83       157

   micro avg       0.77      0.77      0.77       237
   macro avg       0.75      0.75      0.75       237
weighted avg       0.77      0.77      0.77       237


# NEIGHBOURS
Vectorizer: W słowniku znajduje się 35025 różnych słów
- Zbiór treningowy:
	ACCURACY: 99.4026%
              precision    recall  f1-score   support

  ActionName       0

In [5]:
pipeline = Pipeline([          
    ('vectorizer',
     TfidfVectorizer(
         preprocessor=dummy, 
         ngram_range=(1, 5),
         analyzer='word',
         max_df=0.15,
     )),
    ('clf', 
     LogisticRegression(
         class_weight='balanced', # better results for imbalanced Capability - NoCapability
         solver='liblinear' # podobno: best for small datasets
     )
    )
])

do_the_pipeline(pipeline, main_cats, dataframes, accuracy=True, report=True, top_features=False, neigh=False)


>>> ActionName:

# RELATIONS
Vectorizer: W słowniku znajduje się 53614 różnych słów
- Zbiór treningowy:
	ACCURACY: 97.6404%
              precision    recall  f1-score   support

  ActionName       0.96      0.97      0.97      1154
NoActionName       0.99      0.98      0.98      2194

   micro avg       0.98      0.98      0.98      3348
   macro avg       0.97      0.98      0.97      3348
weighted avg       0.98      0.98      0.98      3348

- Zbiór testowy:
	ACCURACY: 78.0591%
              precision    recall  f1-score   support

  ActionName       0.66      0.72      0.69        80
NoActionName       0.85      0.81      0.83       157

   micro avg       0.78      0.78      0.78       237
   macro avg       0.76      0.77      0.76       237
weighted avg       0.79      0.78      0.78       237


>>> Capability:

# RELATIONS
Vectorizer: W słowniku znajduje się 53614 różnych słów
- Zbiór treningowy:
	ACCURACY: 97.9391%
              precision    recall  f1-score   support

  Ca

In [6]:
pipeline = Pipeline([          
    ('vectorizer',
     CountVectorizer(
         preprocessor=dummy, 
         ngram_range=(1, 1),
         analyzer='word'
     )),
    ('clf', 
     MultinomialNB()
    )
])

do_the_pipeline(pipeline, main_cats, dataframes, accuracy=True, report=True, top_features=False, neigh=False)


>>> ActionName:

# RELATIONS
Vectorizer: W słowniku znajduje się 2776 różnych słów
- Zbiór treningowy:
	ACCURACY: 87.3656%
              precision    recall  f1-score   support

  ActionName       0.81      0.83      0.82      1154
NoActionName       0.91      0.90      0.90      2194

   micro avg       0.87      0.87      0.87      3348
   macro avg       0.86      0.86      0.86      3348
weighted avg       0.87      0.87      0.87      3348

- Zbiór testowy:
	ACCURACY: 80.1688%
              precision    recall  f1-score   support

  ActionName       0.70      0.72      0.71        80
NoActionName       0.86      0.84      0.85       157

   micro avg       0.80      0.80      0.80       237
   macro avg       0.78      0.78      0.78       237
weighted avg       0.80      0.80      0.80       237


>>> Capability:

# RELATIONS
Vectorizer: W słowniku znajduje się 2776 różnych słów
- Zbiór treningowy:
	ACCURACY: 91.5771%
              precision    recall  f1-score   support

  Capa

In [7]:
pipeline = Pipeline([          
    ('vectorizer',
     TfidfVectorizer(
         preprocessor=dummy, 
         ngram_range=(1, 3),
         analyzer='word',
         max_features=3000
     )),
    ('clf', 
     MultinomialNB()
    )
])

do_the_pipeline(pipeline, main_cats, dataframes, accuracy=True, report=True, top_features=False, neigh=False)


>>> ActionName:

# RELATIONS
Vectorizer: W słowniku znajduje się 3000 różnych słów
- Zbiór treningowy:
	ACCURACY: 88.1123%
              precision    recall  f1-score   support

  ActionName       0.90      0.73      0.81      1154
NoActionName       0.87      0.96      0.91      2194

   micro avg       0.88      0.88      0.88      3348
   macro avg       0.89      0.85      0.86      3348
weighted avg       0.88      0.88      0.88      3348

- Zbiór testowy:
	ACCURACY: 79.7468%
              precision    recall  f1-score   support

  ActionName       0.79      0.55      0.65        80
NoActionName       0.80      0.92      0.86       157

   micro avg       0.80      0.80      0.80       237
   macro avg       0.79      0.74      0.75       237
weighted avg       0.80      0.80      0.79       237


>>> Capability:

# RELATIONS
Vectorizer: W słowniku znajduje się 3000 różnych słów
- Zbiór treningowy:
	ACCURACY: 89.6953%
              precision    recall  f1-score   support

  Capa

In [18]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([          
    ('vectorizer',
     TfidfVectorizer(
         preprocessor=dummy, 
         ngram_range=(1, 3),
         analyzer='word',
         max_features=3000
     )),
    ('clf', 
     RandomForestClassifier(n_estimators=50)
    )
])

do_the_pipeline(pipeline, main_cats, dataframes, accuracy=True, report=False, top_features=False, neigh=False)


>>> ActionName:

# RELATIONS
Vectorizer: W słowniku znajduje się 3000 różnych słów
- Zbiór treningowy:
	ACCURACY: 99.7909%
- Zbiór testowy:
	ACCURACY: 76.7932%

>>> Capability:

# RELATIONS
Vectorizer: W słowniku znajduje się 3000 różnych słów
- Zbiór treningowy:
	ACCURACY: 99.8805%
- Zbiór testowy:
	ACCURACY: 78.903%


In [ ]:
Tutaj mała próba co by było, gdyby klasyfikować tak po prostu jedno słowo. :P

In [49]:
from nltk.stem.wordnet import WordNetLemmatizer

train = dataframes['Capability']['Train']
test = dataframes['Capability']['Test']

lemma = WordNetLemmatizer()
for dset in [train, test]:
    dset['lem-token'] = dset['token'].apply(lambda x : x.lower())
    dset['lem-token'] = dset['token'].apply(lemma.lemmatize)

vectorizer = CountVectorizer(preprocessor=dummy)
x_counts = vectorizer.fit_transform(train['lem-token'])
x_test_counts = vectorizer.transform(test['lem-token'])

nb = MultinomialNB()
nb.fit(x_counts, train['label_num'])

print(classification_report(test['label_num'], nb.predict(x_test_counts)))

              precision    recall  f1-score   support

           0       0.33      0.02      0.04        47
           1       0.80      0.99      0.89       190

   micro avg       0.80      0.80      0.80       237
   macro avg       0.57      0.51      0.46       237
weighted avg       0.71      0.80      0.72       237

